In [3]:
# Importar librerías para tratamiento de datos
import pandas as pd  

# Importar librerías para web scraping y manipulación de datos
from bs4 import BeautifulSoup
import requests
import re
import random

# Importar librerías para automatización de navegadores web con Selenium
from selenium import webdriver  
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager  
from selenium.webdriver.common.keys import Keys  
from selenium.webdriver.support.ui import Select  

# Importar librerías para pausar la ejecución
from time import sleep  

# Configuraciones
pd.set_option('display.max_columns', None)  # Establece una opción de Pandas para mostrar todas las columnas de un DataFrame.

import warnings
warnings.filterwarnings('ignore')

In [2]:
df_1 = pd.read_csv("INFO_PELIS/cortos_2000_2006.csv", index_col = 0)
df_2 = pd.read_csv("INFO_PELIS/cortos_2021.csv", index_col = 0)
df_3 = pd.read_csv("INFO_PELIS/cortos_2022.csv", index_col = 0)
df_4 = pd.read_csv("INFO_PELIS/cortos_2023.csv", index_col = 0)
df_5 = pd.read_csv("INFO_PELIS/peliculas_2021_2023.csv", index_col = 0)
df_6 = pd.read_csv("INFO_PELIS/pelis_2000_2006.csv", index_col = 0)
df_7 = pd.read_csv("INFO_PELIS/pelis_2007_2010.csv", index_col = 0)
df_8 = pd.read_csv("INFO_PELIS/pelis_2011_2013.csv", index_col = 0)
df_9 = pd.read_csv("INFO_PELIS/cortos_2007_2008.csv", index_col = 0)
df_10 = pd.read_csv("INFO_PELIS/cortos_2009_2010.csv", index_col = 0)
df_11 = pd.read_csv("INFO_PELIS/cortos_2011.csv", index_col = 0)
df_12 = pd.read_csv("INFO_PELIS/cortos_2012.csv", index_col = 0)
df_13 = pd.read_csv("INFO_PELIS/cortos_2013.csv", index_col = 0)
df_14 = pd.read_csv("INFO_PELIS/lista_pelis_2014_2020.csv", index_col = 0)
df_15 = pd.read_csv("INFO_PELIS/cortos_2014_2020.csv", index_col = 0)



In [3]:
df_15.head()

,Genero,Tipo,Titulo,Año de estreno,Mes de estreno,ID
0,Drama,Short,Le sacre d'automne,2014,11.0,tt0165933
1,Drama,Short,Eloise,2014,NaN,tt0839791
2,Drama,Short,Cocoon,2014,8.0,tt10005668
3,Drama,Short,"Zahtevam most, pripada mi!",2014,9.0,tt10010244
4,Drama,Short,Nothing is as it seems,2014,NaN,tt10010624


In [4]:
df_peliculas = pd.concat([df_1,df_2,df_3,df_4,df_5,df_6,df_7,df_8,df_9,df_10,df_11,df_12,df_13,df_14,df_15], axis=0)

In [5]:
df_peliculas.head()

,Genero,Tipo,Titulo,Año de estreno,Mes de estreno,ID
0,Drama,Short,Chateaubriand - Cabeça de Paraíba,2000,12.0,tt0115874
1,Drama,Short,Nights,2000,3.0,tt0138655
2,Drama,Short,Caught,2000,8.0,tt0142155
3,Drama,Short,Smog,2000,3.0,tt0144613
4,Drama,Short,Safe Journey,2000,NaN,tt0188189


In [6]:
df_peliculas.tail()

,Genero,Tipo,Titulo,Año de estreno,Mes de estreno,ID
196698,Action,Short,The Case,2020,1.0,tt31015331
196699,Action,Short,John Cross: Redemption,2020,10.0,tt31123822
196700,Action,Short,Fun Crimes,2020,3.0,tt31171766
196701,Action,Short,Bleeder,2020,6.0,tt31415809
196702,Action,Short,Aadhya,2020,12.0,tt31459299


In [7]:
df_pelis_ordenado = df_peliculas.sort_values(by="Año de estreno")

In [8]:
df_pelis_ordenado.to_csv("pelis_ordenadas.csv")

In [9]:
lista_titulos= df_pelis_ordenado["Titulo"].to_list()

In [10]:
lista_años= df_pelis_ordenado["Año de estreno"].to_list()

In [11]:
list_busqueda_rotten = []
for titulo,año in zip(lista_titulos,lista_años):
    list_busqueda_rotten.append({titulo:año})


In [12]:
lista_tipo= df_pelis_ordenado["Tipo"].to_list()
cortos= 0
pelis = 0

for elemento in lista_tipo:
    if elemento == "Short":
        cortos += 1
    elif elemento == "Movie":
        pelis +=1

print(cortos,pelis)

#30% pelis y 70%cortos

402033 179660


In [13]:
lista_ID = df_pelis_ordenado["ID"].to_list()

In [14]:
lista1=[df_1,df_2,df_3,df_4,df_5,df_6,df_7,df_8,df_9,df_10,df_11,df_12,df_13,df_14,df_15]

suma = 0
for e in lista1:
    suma += len(e)
print("Longitus suma de los data",suma)   
print("Longitud DATAFRAME", len(lista_ID)) 


Longitus suma de los data 581693
Longitud DATAFRAME 581693


In [15]:
driver = webdriver.Chrome()

url = "https://www.imdb.com/?ref_=nv_home"

driver.get(url)

driver.maximize_window()

sleep(5)

# Aceptar las cookies PAG IMDB
driver.find_element("css selector", "#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-dkrFOg.sc-iBYQkv.dcvrLS.ddtuHe.dRCGjd").click()

sleep(5)


## CAMBIAR A TUPLA
diccionario_peliculas = {"Puntuacion_IMDB":[], "Dirección": [],"Guionistas":[], "Argumento":[], "Duración":[], "Nombre de la película": []}
dict_actores={}
# diccionario_actores = {"Actor":[], "Fehca_nacimiento":[], "Conocido_por:":[], "Que_hace":[], "Premios":[]}


contador = 10000
maximo = 13000
intentos = 0 
antiguo_contador = 0 

while contador <13001:

    try:

        for pelicula in lista_ID[contador:maximo+1]:
                        
            #Accedemos al buscador e introduccimos la peli para buscar PAG IMDB
            driver.find_element("css selector",'#suggestion-search').send_keys(pelicula, Keys.ENTER)

            sleep(random.randint(5,12))

            # Seleccionar nombre de la pelicula
            try:
                diccionario_peliculas["Nombre de la película"].append(driver.find_element("css selector", 'span[data-testid="hero__primary-text"]').text)
            except:
                diccionario_peliculas["Nombre de la película"].append(f"Error nombre para la pelicula con id: {pelicula}")
                
            # Seleccionar puntuacion 
            try: 
                diccionario_peliculas["Puntuacion_IMDB"].append(driver.find_element("css selector",'div[data-testid="hero-rating-bar__aggregate-rating__score"] > span:first-child').text)
                
            except:  
                diccionario_peliculas["Puntuacion_IMDB"].append("--")


            existe_direccion = False
            existe_guion = False

            for elemento in driver.find_elements("css selector",'li[data-testid="title-pc-principal-credit"]'):
        
                try:
                    nombre = elemento.find_element("css selector", "span").text
                    if nombre == "Dirección":
                        existe_direccion = True

                        # Seleccionar director
                        try:          
                            diccionario_peliculas["Dirección"].append([e.text for e in elemento.find_elements("css selector",'div > ul > li > a') if e.text != ""])
                        except:
                            diccionario_peliculas["Dirección"].append("DESCONOCIDO") 
                    

                    elif nombre == "Guión":
                        existe_guion = True
                    
                        # Seleccionar Guionista
                        try:          
                            diccionario_peliculas["Guionistas"].append([e.text for e in elemento.find_elements("css selector",'div > ul > li > a') if e.text != ""])
                        except:
                            diccionario_peliculas["Guionistas"].append("DESCONOCIDO")
                except:
                    continue

            if existe_direccion == False:    
                diccionario_peliculas["Dirección"].append("DESCONOCIDO")
            if existe_guion == False:
                diccionario_peliculas["Guionistas"].append("DESCONOCIDO")
            

            sleep(5)
            # Seleccionar ARGUMENTO
            
            try:      
                diccionario_peliculas["Argumento"].append(driver.find_element("css selector", 'span[data-testid="plot-xl"]').text)
                    
            except:
                diccionario_peliculas["Argumento"].append("Desconocido")

            # Seleccionar DURACION        
            try:   
                busqueda_duracion = driver.find_element("css selector",'div.sc-491663c0-3.bdjVSf > div:first-child > ul > li:last-child').text
                if "min" in busqueda_duracion or "h" in busqueda_duracion:
                    diccionario_peliculas["Duración"].append(busqueda_duracion)
                else:
                    diccionario_peliculas["Duración"].append("--")
            except:
                    diccionario_peliculas["Duración"].append("--")

            # Buscar los 10 primeros actores    
            try:
                for elemento in driver.find_elements("css selector",'section[data-testid="title-cast"]'):
                    actor =[e for e in  elemento.find_elements("css selector", "div > div > div > div > a") if e.text != "" and e.text != "\n"]  
            
                    for element in actor[0:10]:
                        nombre = element.text
                    
                        if nombre not in dict_actores.keys():
                            dict_actores[nombre]  = element.get_attribute("href")

                            driver.find_element("css selector",'section[data-testid="title-cast"]')
            
            except:
                    dict_actores[nombre]  = "desconocido"
        
            contador += 1

    except:
        print(f"ha fallado la pelicula {contador}")
        if antiguo_contador == contador:
            intentos += 1
            if intentos == 4:
                sleep(20)
                intentos = 0
                driver.back()
        else:
            intentos += 1
            antiguo_contador = contador



   

### REVISAR:
# Lista de tuplas

ha fallado la pelicula 10084
ha fallado la pelicula 10084
ha fallado la pelicula 10084
ha fallado la pelicula 10084
ha fallado la pelicula 10259
ha fallado la pelicula 10259
ha fallado la pelicula 10259
ha fallado la pelicula 10259
ha fallado la pelicula 10421
ha fallado la pelicula 10421
ha fallado la pelicula 10421
ha fallado la pelicula 10421
ha fallado la pelicula 10518
ha fallado la pelicula 10518
ha fallado la pelicula 10518
ha fallado la pelicula 10518
ha fallado la pelicula 10631
ha fallado la pelicula 10631
ha fallado la pelicula 10631
ha fallado la pelicula 10631
ha fallado la pelicula 10652
ha fallado la pelicula 10652
ha fallado la pelicula 10652
ha fallado la pelicula 10652
ha fallado la pelicula 10666
ha fallado la pelicula 10666
ha fallado la pelicula 10666
ha fallado la pelicula 10666
ha fallado la pelicula 10700
ha fallado la pelicula 10700
ha fallado la pelicula 10700
ha fallado la pelicula 10700
ha fallado la pelicula 10741
ha fallado la pelicula 10741
ha fallado la 

KeyboardInterrupt: 

In [7]:

df_actor = pd.read_csv("union_actores_unicos.csv")
print(df_actor["name"])

0                 Joanna Cole
1         Stanley Wojculewski
2            Melanie Stablein
3                 Max Baloian
4                Keith Davies
                 ...         
167296         Welismar Silva
167297           welton silva
167298          Makoto Kurami
167299           Hiroshi Miki
167300            Ken Musashi
Name: name, Length: 167301, dtype: object


In [10]:
df_actor = pd.read_csv("union_actores_unicos.csv")
dict_actores_limpio= {"name": df_actor["name"].to_list(), "link":[]}

for e in df_actor["link"]:
    try:
        dict_actores_limpio["link"].append(e.replace("'","").split(",")[1])
    except:
        dict_actores_limpio["link"].append(e)


df_actor_limpio = pd.DataFrame(dict_actores_limpio)
df_actor_limpio

In [2]:

df_actor = pd.read_csv("union_actores_unicos.csv")


##Busqueda info actores 
options = webdriver.FirefoxOptions()
options.page_load_strategy = "eager"
driver = webdriver.Firefox(options)
datos_actores= {"Nombre":[], "Nacimiento":[], "Conocido":[], "Que hace":[], "Premios":[], "Oscar": []}

indice_inicio = 0
indice_final = 5
intentos = 0 
antiguo_contador = 0

while indice_inicio <= indice_final:

    try:

        for i in range(len(df_actor))[indice_inicio:indice_final+1]:
            
            print(df_actor.iloc[i]["link"][1])
            driver.get(df_actor.iloc[i]["link"][1])
            driver.maximize_window()
            
            sleep(1)
                # Aceptar las cookies PAG IMDB
            try:
                driver.find_element("css selector", "#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-dkrFOg.sc-iBYQkv.dcvrLS.ddtuHe.dRCGjd").click()
                print("cookies accepted")

            except:
                print("cookies haven't been accepted")

            datos_actores["Nombre"].append(df_actor.iloc[i]["nombre_actor"])

                # Buscar la edad
            try:
                busqueda_anio = driver.find_elements("css selector",'div[data-testid="birth-and-death-birthdate"]:last-child > span:last-child')[-1].text
                anio = busqueda_anio.split()
                datos_actores["Nacimiento"].append(anio[-1])
                
            except:
                datos_actores["Nacimiento"].append("Desconocido")

            #Buscamos porque es conocido

            try:
                conocido_por = driver.find_elements("css selector",'div[data-testid="nm_flmg_kwn_for"] > div > div:nth-child(2) > div > div > div > a')

                lista_conocido = []
                
                for x in conocido_por:
                    lista_conocido.append(x.text)

                datos_actores["Conocido"].append(lista_conocido)
                
            except:
                datos_actores["Conocido"].append("Desconocido")

            #Buscamos que hace
            try:
                profesiones = driver.find_elements("css selector",'ul.ipc-inline-list.ipc-inline-list--show-dividers.sc-d8941411-2.cdJsTz.baseAlt > li')
                lista_profesiones = []

                for profesion in profesiones:
                    lista_profesiones.append(profesion.text)
                
                datos_actores["Que hace"].append(lista_profesiones)

                
            except:
                datos_actores["Que hace"].append("Desconocido")
                
            #Buscamos los prmios
            
            try:
                premios = driver.find_element("css selector",'div[data-testid="awards"] > ul > li > div > ul > li > span').text
                datos_actores["Premios"].append(premios)

            except:
                datos_actores["Premios"].append("Desconocido")

            #Buscamos los prmios OSCAR
            
            try:
                oscars = driver.find_element("css selector",'div[data-testid="awards"] > ul > li > a').text
                if "oscar" not in oscars.lower():
                    datos_actores["Oscar"].append("--")
                else:
                    datos_actores["Oscar"].append(oscars)

            except:
                datos_actores["Oscar"].append("--")
        
            indice_inicio +=1 


    except:
        print(f"ha fallado la pelicula {indice_inicio}")
        if antiguo_contador == indice_inicio:
            intentos += 1
            if intentos == 4:
                sleep(20)
                intentos = 0
                driver.back()
        else:
            intentos += 1
            antiguo_contador = indice_inicio    

    
    
    


'
ha fallado la pelicula 0
'
ha fallado la pelicula 0
'
ha fallado la pelicula 0
'
ha fallado la pelicula 0
'
ha fallado la pelicula 0
'
ha fallado la pelicula 0
'
ha fallado la pelicula 0
'
ha fallado la pelicula 0


In [121]:
## Extraccion datos Rotten Tomatoes

url= "https://www.google.es/"


options = webdriver.FirefoxOptions()
options.page_load_strategy = "eager"
driver = webdriver.Firefox(options)

driver.get(url)

driver.maximize_window()

# Aceptar las cookies de GOOGLE
driver.find_element("css selector", "#L2AGLb > div").click()
sleep(3)

puntuacion_tomatoes = {}

#Establecemos los indices de la lista que queremos recorrer.
contador = 30001
maximo = 48300

for peli in list_busqueda_rotten[contador:maximo]:
    for titulo,año in peli.items():
         
        busqueda = f"TOMATOMETER title: {titulo} - year: {año}"
        # print(busqueda)
        
        try:
                       
            buscador = driver.find_element("css selector",'#APjFqb')
            buscador.clear()
            buscador.send_keys(busqueda, Keys.ENTER)
            sleep(random.randint(4,8))

            try:
                puntuacion_tomatoes[titulo]=driver.find_element("css selector","#rso > div:first-child > div > div > div > div:nth-child(3) > div > span:nth-child(2)").text
                                
            except:
                puntuacion_completa= driver.find_element("css selector",'#rso > div:nth-child(1) > div > div > div:nth-child(3) > div').text
                puntuacion = re.search(r'\b(\d+)%', puntuacion_completa)
                puntuacion_tomatoes[titulo]=puntuacion.group()
        
        except:
            puntuacion_tomatoes[titulo]="No existe puntuacion"



df_tomatoes = pd.Series(puntuacion_tomatoes,name="puntuacion")
df_tomatoes.index.name="titulo"
df_tomatoes.reset_index()
df_tomatoes.to_csv(f"{contador}_{maximo}_puntuacion_tomates_rebe.csv")
